In [1]:
from polls import *
from UKGE import *
import numpy as np

In [2]:
df = pd.read_csv("outputs/resultsclusteredconstituencies.csv")

In [3]:
#Predicting 2019
poll_avg19 = get_weighted_poll_avg(url_19, col_dict=col_dict19)
poll_avg17 = get_weighted_poll_avg(url_17, col_dict=col_dict17)
poll_avg15 = get_weighted_poll_avg(url_15, col_dict=col_dict15)
poll_avg10 = get_weighted_poll_avg(url_10, col_dict=col_dict10)
poll_avg05 = get_weighted_poll_avg(url_05, col_dict=col_dict05)
poll_avg19 = standardise_df(poll_avg19, col_dict19)
poll_avg17 = standardise_df(poll_avg17, col_dict17)
poll_avg15 = standardise_df(poll_avg15, col_dict15)
poll_avg10 = standardise_df(poll_avg10, col_dict10)
poll_avg05 = standardise_df(poll_avg05, col_dict05)



In [4]:

gb19 = get_GB_res(df)
pc_res_19 = convert_to_pc(gb19)
gb17 = get_GB_res(df,yr=2017)
pc_res_17 = convert_to_pc(gb17)
gb15 = get_GB_res(df,yr=2015)
pc_res_15 = convert_to_pc(gb15)
gb10 = get_GB_res(df,yr=2010)
pc_res_10 = convert_to_pc(gb10)
gb05 = get_GB_res(df,yr=2005)
pc_res_05 = convert_to_pc(gb05)

In [5]:
poll_avgs = [poll_avg05,poll_avg10,poll_avg15,poll_avg17,poll_avg19]
pc_results = [pc_res_05,pc_res_10,pc_res_15,pc_res_17,pc_res_19]

In [6]:
bigerrs = []
for p,r in zip(poll_avgs,pc_results):
    conerr = float(p["con"] - r["con"])
    laberr = float(p["lab"] - r["lab"])

    bigerrs.append(conerr)
    bigerrs.append(laberr)






/tmp/ipykernel_250536/1519193469.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  conerr = float(p["con"] - r["con"])
/tmp/ipykernel_250536/1519193469.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  laberr = float(p["lab"] - r["lab"])


In [7]:
big_std = np.std(bigerrs)

In [8]:
pc_res_05

,con,lab,lib,ref,nat,oth
0,0.332274,0.36199,0.226456,0.0,0.022213,0.057067


In [9]:
pd.DataFrame(poll_avg05).transpose().columns

Index(['Con', 'Lab', 'Lib Dem', 'Others', 'con', 'lab', 'lib', 'oth'], dtype='object')

In [10]:
serrs = []
for p,r in zip(poll_avgs,pc_results):
    liberr = float(p["lib"] - r["lib"])
    serrs.append(liberr)
    if "ref" in pd.DataFrame(p).transpose().columns:
        referr = float(p["ref"] - r["ref"])
        serrs.append(referr)
        
    if "nat" in pd.DataFrame(p).transpose().columns:
        naterr = float(p["nat"] - r["nat"])
        serrs.append(naterr)




/tmp/ipykernel_250536/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  liberr = float(p["lib"] - r["lib"])
/tmp/ipykernel_250536/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  liberr = float(p["lib"] - r["lib"])
/tmp/ipykernel_250536/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  liberr = float(p["lib"] - r["lib"])
/tmp/ipykernel_250536/2919726663.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  referr = float(p["ref"] - r["ref"])
/tmp/ipykernel_250536/2919726663.py:3: FutureWarning: Calling float on a single element Series is deprecated and will ra

In [11]:
small_std = np.std(serrs)

In [12]:
constituencies = list(df["New constituency name"])

In [13]:
next_poll_avg = get_weighted_poll_avg(next_url, col_dict=next_col_dict)
nat_polls = standardise_df(next_poll_avg, next_col_dict)
chg_df = nat_polls - pc_res_19
chg_df = chg_df.dropna(axis=1).copy()

In [14]:
chg_df

,con,lab,lib,nat,oth,ref
0,-0.225489,0.109106,-0.03195,-0.015484,0.016707,0.086444


In [15]:
natchgs = []
sims_num = 100
future_uncertainty = 3
for p in std_parties:
    if nat_polls[p] > 0.2:
        natchgs.append(np.random.normal(chg_df[p],big_std*future_uncertainty,sims_num))
    else:
        natchgs.append(np.random.normal(chg_df[p],small_std*future_uncertainty,sims_num))
clusters = list(set(list(df["Cluster"])))
for p in std_parties:
    for c in clusters:
        if nat_polls[p] > 0.2:
            natchgs.append(np.random.normal(0,big_std,sims_num))
        else:
            natchgs.append(np.random.normal(0,small_std,sims_num))

In [16]:
chg_dict = {}


In [17]:
i=0
for p in std_parties:
    chg_dict[p+"n"] = natchgs[i]
    i+=1
for c in clusters:
    for p in std_parties:
        chg_dict[p+str(c)] = natchgs[i]
        i+=1

In [18]:
rand_df = pd.DataFrame(chg_dict)

In [19]:
# Create a dictionary to store the results for each constituency and party
results = {}

# Iterate over constituencies
for con in constituencies:
    # Filter the dataframe for the current constituency
    con_df = df[df["New constituency name"] == con].copy()
    
    # Get the cluster, imputation uncertainty, and future uncertainty for the current constituency
    cluster = con_df["Cluster"].iloc[0]
    imputation_uncertainty = con_df["Low_Confidence_Imputation"].iloc[0]
    
    # Iterate over standard parties
    for p in std_parties:
        # Get the previous results for the current party and constituency
        prev_res = con_df["2019_Results"+p+"_pc"]
        
        # Get the necessary data from the random dataframe
        nat_chg = rand_df[p+"n"]
        clr_chg = rand_df[p+str(cluster)]
        
        # Calculate the noise for the current constituency and party
        con_noise = [np.random.normal(0, (1+imputation_uncertainty)*small_std) for n in range(sims_num)]
        
        # Calculate the rough prediction for the current constituency and party
        prev_res_array = [float(prev_res.iloc[0]) for n in range(sims_num)]
        rough_pred = prev_res_array + nat_chg + clr_chg + con_noise
        rough_pred[rough_pred < 0] = 0
        
        # Store the results in the dictionary
        results[con+p+"Noise"] = con_noise
        results[con+p+"Rough"] = rough_pred
    
# Update the random dataframe with the results
rand_df = pd.concat([rand_df,pd.DataFrame(results)],axis=1)

In [20]:
def get_winner(party_values, winner_val,parties = std_parties):
    for p,v in zip(parties,list(party_values)[0]):
        if v == winner_val:
            return p

final_dfs = [rand_df]
for con in constituencies:
    con_cols = []
    for p in std_parties:
        con_cols.append(con+p+"Rough")
    final_df = rand_df[con_cols].copy()
    final_pred_cols = []
    for p in std_parties:
        final_df[con+p+"Final"] = final_df[con+p+"Rough"] / final_df[con_cols].sum(axis=1)
        final_pred_cols.append(con+p+"Final")
    final_df[con +"past_post_val"] = final_df[final_pred_cols].max(axis=1)
    final_pred_cols.append(con +"past_post_val")
    final_df[con+"Winner"] = final_df[final_pred_cols].apply(lambda x:get_winner([x[final_pred_cols]],x[con+"past_post_val"]),axis=1)
                
    # final_df[con+"Winner"] = winners
    final_dfs.append(final_df)
rand_df = pd.concat(final_dfs, axis=1)
    

In [21]:
seats_cols = []
for p in std_parties:
    rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
    seats_cols.append(p+"Seats")

/tmp/ipykernel_250536/4278369753.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
/tmp/ipykernel_250536/4278369753.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df[p+"Seats"] = rand_df[[c + "Winner" for c in constituencies]].eq(p).sum(1)
/tmp/ipykernel_250536/4278369753.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joini

In [22]:
rand_df[seats_cols]

,conSeats,labSeats,libSeats,refSeats,natSeats,othSeats
0,214,336,18,2,60,2
1,179,382,31,0,39,1
2,104,432,46,0,48,2
3,244,336,14,0,37,1
4,216,343,25,0,47,1
...,...,...,...,...,...,...
95,127,455,17,0,31,2
96,205,354,16,0,55,2
97,177,368,32,0,54,1
98,36,541,23,0,31,1


In [24]:
max_seats = rand_df[seats_cols].max(1)
results = []
for n in range(sims_num):
    for col in seats_cols:
        if max_seats[n] ==rand_df[col].loc[n]:
            for p in std_parties:
                if p + "Seats" == col:
                    winning_party = p
    
    if max_seats[n] <326:
        result = "Hung Parliament: Largest Party " + winning_party
    else:
        result = winning_party
    results.append(result)
rand_df["Result"] = results


/tmp/ipykernel_250536/3465883602.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rand_df["Result"] = results


In [25]:
rand_df["Result"].value_counts()

Result
lab                                   77
Hung Parliament: Largest Party lab    20
Hung Parliament: Largest Party con     3
Name: count, dtype: int64

In [27]:
med_df = rand_df[rand_df["conSeats"]==rand_df["conSeats"].median()]

In [28]:
typical_run = med_df[(med_df["labSeats"]>med_df["labSeats"].quantile(0.4))&(med_df["labSeats"]<med_df["labSeats"].quantile(0.6))].copy()

In [29]:
typical_run

,conn,labn,libn,refn,natn,othn,con0,lab0,lib0,ref0,...,West DunbartonshireothFinal,West Dunbartonshirepast_post_val,West DunbartonshireWinner,conSeats,labSeats,libSeats,refSeats,natSeats,othSeats,Result


In [30]:
print(nat_polls["lab"] - nat_polls["con"])

0.21383333333333324


In [31]:
nat_polls

Con         0.234500
Lab         0.448333
Lib Dems    0.089833
SNP         0.030500
Green       0.061167
Reform      0.107667
Others      0.028500
con         0.234500
lab         0.448333
lib         0.089833
ref         0.107667
nat         0.030500
oth         0.028500
dtype: float64

In [32]:
winner_cols = [con+"Winner" for con in constituencies]

In [33]:
winner_wins = []
for col in winner_cols:
    winner_wins.append(max(list(rand_df[col].value_counts())))

In [34]:
close_cols = []
for col,wins in zip(winner_cols,winner_wins):
    if wins < 525:
        close_cols.append(col)

In [35]:
rand_df[close_cols]

,Aberafan MaestegWinner,AldershotWinner,Aldridge-BrownhillsWinner,Altrincham and Sale WestWinner,Alyn and DeesideWinner,Amber ValleyWinner,Arundel and South DownsWinner,AshfieldWinner,AshfordWinner,Ashton-under-LyneWinner,...,North Ayrshire and ArranWinner,North East FifeWinner,Orkney and ShetlandWinner,Paisley and Renfrewshire NorthWinner,Paisley and Renfrewshire SouthWinner,Perth and Kinross-shireWinner,RutherglenWinner,Stirling and StrathallanWinner,West Aberdeenshire and KincardineWinner,West DunbartonshireWinner
0,lab,con,con,lab,lab,con,con,oth,con,lab,...,nat,nat,nat,nat,nat,nat,nat,nat,nat,nat
1,lab,con,con,lab,lab,con,con,lab,con,lab,...,nat,lib,lib,nat,nat,nat,lab,nat,nat,nat
2,lab,lab,con,lab,lab,lab,con,lab,con,lab,...,nat,nat,lib,nat,nat,nat,nat,nat,nat,nat
3,lab,con,con,lab,lab,con,con,lab,con,lab,...,nat,nat,lib,nat,nat,nat,lab,nat,nat,nat
4,lab,con,con,lab,lab,con,con,lab,con,lab,...,nat,nat,lib,nat,nat,nat,nat,nat,nat,nat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,lab,lab,con,lab,lab,lab,con,lab,con,lab,...,nat,nat,lib,lab,lab,nat,lab,nat,nat,lab
96,lab,con,con,lab,lab,con,con,lab,con,lab,...,nat,nat,nat,nat,nat,nat,nat,nat,nat,nat
97,lab,con,con,lab,lab,con,con,lab,lab,lab,...,nat,nat,lib,nat,nat,nat,nat,nat,nat,nat
98,lab,lab,lab,lab,lab,lab,lab,lab,lab,lab,...,nat,nat,lib,lab,lab,nat,lab,nat,nat,lab


In [36]:
rand_df

,conn,labn,libn,refn,natn,othn,con0,lab0,lib0,ref0,...,West DunbartonshireothFinal,West Dunbartonshirepast_post_val,West DunbartonshireWinner,conSeats,labSeats,libSeats,refSeats,natSeats,othSeats,Result
0,-0.223429,0.055543,-0.102301,0.161835,0.031092,0.034055,0.018101,-0.008949,-0.014688,-0.034052,...,0.065844,0.476531,nat,214,336,18,2,60,2,lab
1,-0.230719,0.115823,-0.010287,0.043983,-0.058138,0.047916,0.024663,-0.034290,0.006416,-0.010180,...,0.043567,0.424059,nat,179,382,31,0,39,1,lab
2,-0.312989,0.102839,-0.027478,0.127807,-0.019218,-0.028354,-0.037144,0.008080,0.004980,0.013801,...,0.000000,0.488734,nat,104,432,46,0,48,2,lab
3,-0.116529,0.144269,-0.077180,0.075734,-0.039386,0.082036,-0.053858,0.001084,-0.008186,0.011161,...,0.099517,0.423705,nat,244,336,14,0,37,1,lab
4,-0.220955,0.054648,-0.071782,0.093413,-0.087128,0.026332,-0.005640,0.000074,-0.012285,-0.042063,...,0.017570,0.450643,nat,216,343,25,0,47,1,lab
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.204063,0.194663,-0.087602,0.026076,-0.048665,0.063582,-0.019414,0.023258,-0.023647,-0.018654,...,0.068641,0.481125,lab,127,455,17,0,31,2,lab
96,-0.213369,0.100716,-0.108649,0.068942,0.038728,0.046494,-0.023309,-0.023331,-0.046338,0.026530,...,0.042204,0.501903,nat,205,354,16,0,55,2,lab
97,-0.242332,0.091309,-0.026523,0.147483,0.013851,0.002463,-0.016321,0.012786,0.018753,0.013210,...,0.030662,0.473288,nat,177,368,32,0,54,1,lab
98,-0.291715,0.202034,-0.051402,0.039647,-0.000014,-0.025974,-0.044259,0.044493,-0.033677,-0.001184,...,0.000000,0.496208,lab,36,541,23,0,31,1,lab
